## Definicion de funciones

In [ ]:
# #Convierte grado educacion y grado minimo en variable numerica ordenada con un diccionario
# def conv_var(user1, jobs1):
# 	dict_grad = {'Educacion secundaria obligatoria': 0,
# 				'FP basica': 1,
# 				'FP grado medio': 2,
# 				'Bachillerato': 3,
# 				'FP grado superior': 4,
# 				'Universitario': 5}

# 	user1['grado_educacion'] = dict_grad[user1['grado_educacion']]
# 	jobs1['grado_int'] = jobs1['grado_minimo'].map(dict_grad)

In [ ]:
# #Chequea si hay vacantes en el registro
# def check_vacancies(job):
# 	if job['vacantes'] <= 0:
# 		return (0)
# 	return (1)

In [ ]:
# #Chequea si area y grado de educacion son compatibles para filtrar Dataframe jobs
# def first_filter(user, job, valid_jobs):
# 	if user['area'] == job['area'] and user['grado_educacion'] >= job['grado_int']:
# 		valid_jobs.append(job.id_proyecto)

# 	filter_jobs = jobs[jobs.id_proyecto.isin(valid_jobs)].copy()
# 	filter_jobs.reset_index(inplace=True, drop=True)
# 	return (filter_jobs)

In [ ]:
# #Calcula la similitud jaccard entre dos listas
# def jaccard(x, y):
#     intersection = len(set.intersection(*[set(x), set(y)]))
#     union = len(set.union(*[set(x), set(y)]))
#     return intersection / float(union)

In [ ]:
# #Chequea similitud jaccard entre las habilidades y añade el resultado al Dataframe
# def skills_comparison(user, filter_jobs):
# 	sim_jaccard = []
# 	skills_user = user.habilidades.split(', ')

# 	for i in range(len(filter_jobs)):
# 		job = filter_jobs.loc[i,:]
# 		skills_job = job.habilidades.replace('\n','').split(',')
# 		sim_jaccard.append(jaccard(skills_user, skills_job))

# 	filter_jobs['sim_jaccard'] = sim_jaccard

In [ ]:
# #Preprocesado del texto: convierte todos los caracteres a minuscula, quita los numeros y caracteres especiales
# def string_preprocessor(string_arg):
#     string_arg = string_arg.lower()
#     string_arg = re.sub(r'\d+', '', string_arg)
#     string_arg = re.sub(r'[^\w\s]', '', string_arg)
#     return (string_arg)

In [ ]:
# #Compara el contenido mediante TF-IDF y Cosine Similarity uniendo las variables mas informativas y añade el resultado al Dataframe
# #Aplica lematizacion al texto usando spaCy
# def content_comparison(user, filter_jobs):
# 	sim_cosine = []
# 	nlp = spacy.load('es_core_news_sm')
# 	tf_idf = TfidfVectorizer(stop_words=stop_words, preprocessor=string_preprocessor)
# 	user['infor'] = user['nombre_grado'] + ' ' + user['master'] + ' ' + user['cursos'] + ' ' + user['habilidades']
# 	doc = nlp(user.infor)
# 	user['infor'] = ' '.join([token.lemma_ for token in doc])

# 	for i in range(len(filter_jobs)):
# 		job = filter_jobs.loc[i,:].copy()
# 		job['infor'] = job['perfil'] + ' ' + job['descripcion'] + ' ' + job['habilidades']
# 		doc = nlp(job.infor)
# 		job['infor'] = ' '.join([token.lemma_ for token in doc])
# 		combined_infor = [user.infor, job.infor]
# 		tfidf_matrix = tf_idf.fit_transform(combined_infor)
# 		similarity_tfidf = cosine_similarity(tfidf_matrix)
# 		sim_cosine.append(similarity_tfidf[0][1])

# 	filter_jobs['sim_cosine'] = sim_cosine

In [ ]:
# #Inicia la comparacion
# def init_comparison(user, jobs):
# 	valid_jobs = []
# 	conv_var(user, jobs)

# 	for i in range(len(jobs)):
# 		job = jobs.loc[i,:]
# 		if check_vacancies(job) == 1:
# 			filter_jobs = first_filter(user, job, valid_jobs)

# 	skills_comparison(user, filter_jobs)
# 	content_comparison(user, filter_jobs)
# 	return (filter_jobs)

In [ ]:
# #Entrega la recomendacion final con los k registros mas similares
# def final_recommendation(filter_jobs, k):
# 	filter_jobs['score'] = filter_jobs['sim_jaccard']+filter_jobs['sim_cosine']
# 	recommended_jobs = filter_jobs[filter_jobs.score != 0.0].sort_values(by="score", ascending=False)

# 	if (len(filter_jobs)) > k:
# 		recommended_jobs = recommended_jobs[:k]
# 	return (recommended_jobs)

----------------------------------------
## Script del algoritmo de recomendacion

In [9]:
#------------------------------CONDICIONES INICIALES------------------------------

import numpy as np
import pandas as pd
import re
import spacy
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

df_users = pd.read_csv('Dataset_estudiantes.csv')
df_jobs = pd.read_csv('Dataset_proyectos.csv')

users = df_users[['id_estudiante','area','grado_educacion','nombre_grado','master','cursos','habilidades']]
jobs = df_jobs[['id_proyecto','vacantes','area','grado_minimo','perfil','descripcion','habilidades']].copy()

#------------------------------ALGORITMO DE RECOMENDACION A USER------------------------------

#Convierte grado educacion y grado minimo en variable numerica ordenada con un diccionario
def conv_var(user1, jobs1):
	dict_grad = {'Educacion secundaria obligatoria': 0,
				'FP basica': 1,
				'FP grado medio': 2,
				'Bachillerato': 3,
				'FP grado superior': 4,
				'Universitario': 5}

	user1['grado_educacion'] = dict_grad[user1['grado_educacion']]
	jobs1['grado_int'] = jobs1['grado_minimo'].map(dict_grad)

#Chequea si hay vacantes en el registro
def check_vacancies(job):
	if job['vacantes'] <= 0:
		return (0)
	return (1)

#Chequea si area y grado de educacion son compatibles para filtrar Dataframe jobs
def first_filter(user, job, valid_jobs):
	if user['area'] == job['area'] and user['grado_educacion'] >= job['grado_int']:
		valid_jobs.append(job.id_proyecto)

	filter_jobs = jobs[jobs.id_proyecto.isin(valid_jobs)].copy()
	filter_jobs.reset_index(inplace=True, drop=True)
	return (filter_jobs)

#Calcula la similitud jaccard entre dos listas
def jaccard(x, y):
    intersection = len(set.intersection(*[set(x), set(y)]))
    union = len(set.union(*[set(x), set(y)]))
    return intersection / float(union)

#Chequea similitud jaccard entre las habilidades y añade el resultado al Dataframe
def skills_comparison(user, filter_jobs):
	sim_jaccard = []
	skills_user = user.habilidades.split(', ')

	for i in range(len(filter_jobs)):
		job = filter_jobs.loc[i,:]
		skills_job = job.habilidades.replace('\n','').split(',')
		sim_jaccard.append(jaccard(skills_user, skills_job))

	filter_jobs['sim_jaccard'] = sim_jaccard

#Preprocesado del texto: convierte todos los caracteres a minuscula, quita los numeros y caracteres especiales
def string_preprocessor(string_arg):
    string_arg = string_arg.lower()
    string_arg = re.sub(r'\d+', '', string_arg)
    string_arg = re.sub(r'[^\w\s]', '', string_arg)
    return (string_arg)

#Compara el contenido mediante TF-IDF y Cosine Similarity uniendo las variables mas informativas y añade el resultado al Dataframe
#Aplica lematizacion al texto usando spaCy
def content_comparison(user, filter_jobs):
	sim_cosine = []
	nlp = spacy.load('es_core_news_sm')
	tf_idf = TfidfVectorizer(stop_words=stop_words, preprocessor=string_preprocessor)
	user['infor'] = user['nombre_grado'] + ' ' + user['master'] + ' ' + user['cursos'] + ' ' + user['habilidades']
	doc = nlp(user.infor)
	user['infor'] = ' '.join([token.lemma_ for token in doc])

	for i in range(len(filter_jobs)):
		job = filter_jobs.loc[i,:].copy()
		job['infor'] = job['perfil'] + ' ' + job['descripcion'] + ' ' + job['habilidades']
		doc = nlp(job.infor)
		job['infor'] = ' '.join([token.lemma_ for token in doc])
		combined_infor = [user.infor, job.infor]
		tfidf_matrix = tf_idf.fit_transform(combined_infor)
		similarity_tfidf = cosine_similarity(tfidf_matrix)
		sim_cosine.append(similarity_tfidf[0][1])

	filter_jobs['sim_cosine'] = sim_cosine

#Inicia la comparacion
def init_comparison(user, jobs):
	valid_jobs = []
	conv_var(user, jobs)

	for i in range(len(jobs)):
		job = jobs.loc[i,:]
		if check_vacancies(job) == 1:
			filter_jobs = first_filter(user, job, valid_jobs)

	skills_comparison(user, filter_jobs)
	content_comparison(user, filter_jobs)
	return (filter_jobs)

#Entrega la recomendacion final con los k registros mas similares
def final_recommendation(filter_jobs, k):
	filter_jobs['score'] = filter_jobs['sim_jaccard']+filter_jobs['sim_cosine']
	recommended_jobs = filter_jobs[filter_jobs.score != 0.0].sort_values(by="score", ascending=False)

	if (len(filter_jobs)) > k:
		recommended_jobs = recommended_jobs[:k]
	return (recommended_jobs)

#Main
def main(user):
	filter_jobs = init_comparison(user, jobs)
	recommended_jobs = final_recommendation(filter_jobs, 5)
	display(pd.DataFrame(user).T)
	display(recommended_jobs)
	return (list(recommended_jobs.id_proyecto))

# if __name__ == "__main__":
# 	main(user)

---------------------------------
## Resultados del algoritmo y stopwords

In [10]:
#Visualizacion de las tablas del TF-IDF y Cosine Similarity para user y job especificado
def content_comparison_visualization(idx_user,idx_job):
	pd.set_option("display.max_columns", 120)

	user = users.loc[idx_user, :].copy()
	job = jobs.loc[idx_job, :].copy()
	nlp = spacy.load('es_core_news_sm')
	user['infor'] = user['nombre_grado'] + ' ' + user['master'] + ' ' + user['cursos'] + ' ' + user['habilidades']
	job['infor'] = job['infor'] = job['perfil'] + ' ' + job['descripcion'] + ' ' + job['habilidades']
	doc1 = nlp(user.infor)
	user['infor'] = ' '.join([token.lemma_ for token in doc1])
	doc2 = nlp(job.infor)
	job['infor'] = ' '.join([token.lemma_ for token in doc2])

	combined_infor = [user.infor, job.infor]

	tf_idf = TfidfVectorizer(stop_words=stop_words, preprocessor=string_preprocessor)
	tfidf_matrix = tf_idf.fit_transform(combined_infor)
	args_bag_of_words = tfidf_matrix.toarray()

	col_args = [tup[0] for tup in
							sorted(tf_idf.vocabulary_.items(), key=lambda x: x[1])]
	BoW = pd.DataFrame(args_bag_of_words,
												columns=col_args,
												index=['Estudiante','Proyecto'])
	display(BoW)

	similarity_tfidf = cosine_similarity(BoW)
	matriz_similaridades_df = pd.DataFrame(similarity_tfidf,
											index=combined_infor,
											columns=combined_infor)
	display(matriz_similaridades_df)

In [11]:
# #Este bucle simula la entrada al algoritmo de cada usuario en el Dataframe users
# for i in range(len(users)):
# 	user = users.loc[i,:].copy()
# 	main(user)

#Evalua el resultado para un user generado aleatoriamente
idx_user = np.random.randint(len(users))
user = users.loc[idx_user, :].copy()
id_job = main(user)[0]
idx_job = jobs[jobs['id_proyecto'] == id_job].index[0]
content_comparison_visualization(idx_user,idx_job)

,id_estudiante,area,grado_educacion,nombre_grado,master,cursos,habilidades,infor
368,id_estudiante369,Administración y Gestión,2,Técnico en Gestión Administrativa,Sin máster,Gestión comercial y técnica de seguros y rease...,"Experiencia con sistemas CRM, Adaptabilidad pa...",Técnico en Gestión Administrativa Sin máster G...


,id_proyecto,vacantes,area,grado_minimo,perfil,descripcion,habilidades,grado_int,sim_jaccard,sim_cosine,score
11,id_Proyecto_291,3,Administración y Gestión,FP grado medio,GESTIÓN DE ADMINISTRACIÓN,¡Únete a nuestro equipo como Ingeniero/a Técni...,Conocimiento técnico en obras de urbanización ...,2,0.095238,0.248865,0.344103
12,id_Proyecto_302,1,Administración y Gestión,FP basica,GESTIÓN DE ADMINISTRACIÓN,¡Únete a nuestro equipo como Ingeniero/a Técni...,Conocimiento técnico en obras de urbanización ...,1,0.095238,0.248865,0.344103
13,id_Proyecto_326,1,Administración y Gestión,FP basica,GESTIÓN DE ADMINISTRACIÓN,¡Únete a nuestro equipo como Ingeniero/a Técni...,Conocimiento técnico en obras de urbanización ...,1,0.095238,0.248865,0.344103
22,id_Proyecto_494,1,Administración y Gestión,FP grado medio,TÉCNICO DE ADMINISTRACIÓN DE PERSONAL (GESTIÓN...,Importante Firma de Servicios Profesionales en...,"Dominio de A3Nom y otro software contables,\n ...",2,0.000000,0.268662,0.268662
4,id_Proyecto_100,1,Administración y Gestión,FP basica,ADMINISTRACIÓN Y GESTIÓN EMPRESARIAL,Estamos en la búsqueda de profesional en Admin...,"Gestión contable avanzada, incluyendo la elabo...",1,0.000000,0.261532,0.261532


,accesibilidad,actitud,actor,adaptabilidad,administración,administrativa,administrativo,afección,anclaje,anti,análisis,ascensor,asistemática,asumir,calidad,capacidad,cepo,colaboración,comercial,competencia,complejidad,compliance,comunicación,conocimiento,construcción,contable,contribuir,control,coordinación,coordinar,correspondiente,crm,cumplimiento,cumplir,desarrollo,detalle,diseño,distinto,diversa,diverso,documentación,dominio,efectivo,eficiente,ejecución,elaboración,emisión,entorno,equipo,escrito,espacio,establecido,euskera,evaluación,excelente,experiencia,familiaridad,fiscal,garantizar,gestión,...,lectura,liderar,logro,metodologíos,multidisciplinario,municipal,máster,necesario,normativa,normativo,obra,obras,oral,organización,orientación,orientado,pago,petición,pl,plazo,posición,preceptivo,precisión,previo,privado,proactivo,problema,promotor,proveedor,proyecto,puesto,públicas,público,realización,reaseguro,regulación,regulatory,relacionado,remesa,requisito,resolver,responsabilidad,responsable,resultado,scrum,sección,seguimiento,seguro,servicio,sistema,software,sondeo,tarea,termita,técnico,urbanización,urbanístico,valorar,ágil,únete
Estudiante,0.00000,0.000000,0.000000,0.106991,0.000000,0.150372,0.150372,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.150372,0.00000,0.106991,0.150372,0.000000,0.106991,0.106991,0.150372,0.000000,0.000000,0.00000,0.00000,0.00000,0.150372,0.150372,0.000000,0.000000,0.000000,0.000000,0.000000,0.106991,0.000000,0.150372,0.000000,0.000000,0.00000,0.000000,0.000000,0.150372,0.000000,0.000000,0.000000,0.106991,0.000000,0.00000,0.000000,0.000000,0.213982,0.150372,0.150372,0.000000,0.320973,...,0.150372,0.000000,0.000000,0.150372,0.000000,0.000000,0.150372,0.000000,0.106991,0.150372,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.150372,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.106991,0.00000,0.000000,0.000000,0.150372,0.106991,0.00000,0.000000,0.106991,0.00000,0.150372,0.106991,0.150372,0.106991,0.150372,0.000000,0.000000,0.00000,0.000000,0.000000,0.150372,0.000000,0.000000,0.150372,0.000000,0.150372,0.000000,0.000000,0.000000,0.000000,0.320973,0.000000,0.000000,0.000000,0.150372,0.000000
Proyecto,0.10059,0.050295,0.050295,0.035785,0.050295,0.000000,0.000000,0.050295,0.050295,0.050295,0.050295,0.050295,0.050295,0.050295,0.050295,0.150885,0.050295,0.10059,0.000000,0.10059,0.035785,0.000000,0.050295,0.107356,0.071570,0.000000,0.050295,0.150885,0.10059,0.10059,0.10059,0.000000,0.000000,0.050295,0.050295,0.050295,0.050295,0.050295,0.035785,0.050295,0.000000,0.050295,0.050295,0.10059,0.050295,0.150885,0.000000,0.050295,0.050295,0.050295,0.071570,0.050295,0.10059,0.050295,0.050295,0.107356,0.000000,0.000000,0.050295,0.107356,...,0.000000,0.050295,0.050295,0.000000,0.050295,0.150885,0.000000,0.050295,0.035785,0.000000,0.352064,0.050295,0.050295,0.050295,0.050295,0.050295,0.000000,0.050295,0.10059,0.050295,0.050295,0.10059,0.050295,0.050295,0.286282,0.10059,0.050295,0.050295,0.000000,0.286282,0.10059,0.050295,0.107356,0.10059,0.000000,0.035785,0.000000,0.107356,0.000000,0.050295,0.050295,0.10059,0.050295,0.050295,0.000000,0.050295,0.201179,0.000000,0.150885,0.000000,0.050295,0.050295,0.050295,0.050295,0.143141,0.251474,0.050295,0.050295,0.000000,0.050295


,"Técnico en Gestión Administrativa Sin máster Gestión comercial y técnico de seguro y reaseguro privado Experiencia con sistema CRM , Adaptabilidad para trabajar en proyecto de diversa indo él y complejidad , Conocimiento de normativa y regulación relacionado con el construcción y el espacio público , Habilidad en el gestión de pago a proveedor y emisión de remesa , Experiencia en metodologíos ágil como Scrum o Kanban , Cumplimiento normativo administrativo , fiscal y contable ( Regulatory Compliance ) , Familiaridad con el lectura e interpretación de documentación técnico","gestión DE ADMINISTRACIÓN ¡ únete a nuestro equipo como Ingeniero / a Técnico / a de Obras Públicas y contribuir al desarrollo de proyecto urbanístico de iniciativa privado ! En este posición , serás responsable del control y seguimiento de el obra de urbanización , coordinar tanto proyecto privado como público en colaboración con el servicio municipal correspondiente . \n responsabilidad : \n Control y seguimiento de el obra de urbanización de iniciativa privado asistemática . \n Coordinación de el obra privado con el distinto servicio municipal , en colaboración con el Jefatura de Sección . \n Realización de informe y seguimiento de obra de urbanización de iniciativa privado y obra de instalación de ascensor y otro de accesibilidad con afección al espacio público . \n Elaboración de informe sobre petición de promotor privado relacionado con el ejecución de sondeo , anclaje , cepo anti termita y otro . \n Preceptivo conocimiento de PL Euskera . \n él valorar el experiencia previo en el puesto . Conocimiento técnico en obra de urbanización y construcción , \n Capacidad para el control y seguimiento efectivo de proyecto de iniciativa privado , \n Habilidad para coordinar proyecto privado con el servicio municipal correspondiente , \n Experiencia en el elaboración de informe técnico sobre obra de urbanización y accesibilidad , \n Conocimiento de normativa y regulación relacionado con el construcción y el espacio público , \n Competencia en el realización de análisis y evaluación técnico de proyecto , \n Habilidad para trabajar de manera eficiente en entorno multidisciplinario , \n Dominio del idioma PL Euskera requisito preceptivo para el puesto , \n excelente habilidad de comunicación oral y escrito para el elaboración de informe y el coordinación con diverso actor , \n Capacidad para resolver problema de manera proactivo y eficiente , \n Experiencia en el uso de herramienta y software de ingeniería relacionado con el diseño y el gestión de proyecto , \n Competencia en el gestión del tiempo y el organización de tarea para cumplir con el plazo establecido , \n Adaptabilidad para trabajar en proyecto de diversa indo él y complejidad , \n Orientación al detalle para garantizar el precisión y calidad en el trabajo realizado , \n Actitud proactivo y orientado al logro de resultado con capacidad para asumir responsabilidad y liderar proyecto cuando ser necesario \n"
"Técnico en Gestión Administrativa Sin máster Gestión comercial y técnico de seguro y reaseguro privado Experiencia con sistema CRM , Adaptabilidad para trabajar en proyecto de diversa indo él y complejidad , Conocimiento de normativa y regulación relacionado con el construcción y el espacio público , Habilidad en el gestión de pago a proveedor y emisión de remesa , Experiencia en metodologíos ágil como Scrum o Kanban , Cumplimiento normativo administrativo , fiscal y contable ( Regulatory Compliance ) , Familiaridad con el lectura e interpretación de documentación técnico",1.000000,0.248865
"gestión DE ADMINISTRACIÓN ¡ únete a nuestro equipo como Ingeniero / a Técnico / a de Obras Públicas y contribuir al desarrollo de proyecto urbanístico de iniciativa privado ! En este posición , serás responsable del control y seguimiento de el obra de urbanización , coordinar tanto proyecto privado como público en colaboración con el servicio municipal correspondiente . \n responsabilidad : \n Control y seguimie

In [ ]:
stop_words = '''a
actualmente
adelante
además
afirmó
agregó
ahora
ahí
al
algo
alguna
algunas
alguno
algunos
algún
alrededor
ambos
ampleamos
ante
anterior
antes
apenas
aproximadamente
aquel
aquellas
aquellos
aqui
aquí
arriba
aseguró
así
atras
aunque
ayer
añadió
aún
bajo
bastante
bien
buen
buena
buenas
bueno
buenos
cada
casi
cerca
cierta
ciertas
cierto
ciertos
cinco
comentó
como
con
conocer
conseguimos
conseguir
considera
consideró
consigo
consigue
consiguen
consigues
contra
cosas
creo
cual
cuales
cualquier
cuando
cuanto
cuatro
cuenta
cuya
cómo
da
dado
dan
dar
de
debe
deben
debido
decir
dejó
del
demás
dentro
desde
después
dice
dicen
dicho
dieron
diferente
diferentes
dijeron
dijo
dio
donde
dos
durante
e
ejemplo
el
ella
ellas
ello
ellos
embargo
empleais
emplean
emplear
empleas
empleo
en
encima
encuentra
entonces
entre
era
erais
eramos
eran
eras
eres
es
esa
esas
ese
eso
esos
esta
estaba
estabais
estaban
estabas
estad
estada
estadas
estado
estados
estais
estamos
estan
estando
estar
estaremos
estará
estarán
estarás
estaré
estaréis
estaría
estaríais
estaríamos
estarían
estarías
estas
este
estemos
esto
estos
estoy
estuve
estuviera
estuvierais
estuvieran
estuvieras
estuvieron
estuviese
estuvieseis
estuviesen
estuvieses
estuvimos
estuviste
estuvisteis
estuviéramos
estuviésemos
estuvo
está
estábamos
estáis
están
estás
esté
estéis
estén
estés
ex
existe
existen
explicó
expresó
fin
fue
fuera
fuerais
fueran
fueras
fueron
fuese
fueseis
fuesen
fueses
fui
fuimos
fuiste
fuisteis
fuéramos
fuésemos
gran
grandes
gueno
ha
haber
habida
habidas
habido
habidos
habiendo
habremos
habrá
habrán
habrás
habré
habréis
habría
habríais
habríamos
habrían
habrías
habéis
había
habíais
habíamos
habían
habías
hace
haceis
hacemos
hacen
hacer
hacerlo
haces
hacia
haciendo
hago
han
has
hasta
hay
haya
hayamos
hayan
hayas
hayáis
he
hecho
hemos
hicieron
hizo
hoy
hube
hubiera
hubierais
hubieran
hubieras
hubieron
hubiese
hubieseis
hubiesen
hubieses
hubimos
hubiste
hubisteis
hubiéramos
hubiésemos
hubo
igual
incluso
indicó
informó
intenta
intentais
intentamos
intentan
intentar
intentas
intento
ir
junto
la
lado
largo
las
le
les
llegó
lleva
llevar
lo
los
luego
lugar
manera
manifestó
mayor
me
mediante
mejor
mencionó
menos
mi
mientras
mio
mis
misma
mismas
mismo
mismos
modo
momento
mucha
muchas
mucho
muchos
muy
más
mí
mía
mías
mío
míos
nada
nadie
ni
ninguna
ningunas
ninguno
ningunos
ningún
no
nos
nosotras
nosotros
nuestra
nuestras
nuestro
nuestros
nueva
nuevas
nuevo
nuevos
nunca
o
ocho
os
otra
otras
otro
otros
para
parece
parte
partir
pasada
pasado
pero
pesar
poca
pocas
poco
pocos
podeis
podemos
poder
podria
podriais
podriamos
podrian
podrias
podrá
podrán
podría
podrían
poner
por
por qué
porque
posible
primer
primera
primero
primeros
principalmente
propia
propias
propio
propios
próximo
próximos
pudo
pueda
puede
pueden
puedo
pues
que
quedó
queremos
quien
quienes
quiere
quién
qué
realizado
realizar
realizó
respecto
sabe
sabeis
sabemos
saben
saber
sabes
se
sea
seamos
sean
seas
segunda
segundo
según
seis
ser
seremos
será
serán
serás
seré
seréis
sería
seríais
seríamos
serían
serías
seáis
señaló
si
sido
siempre
siendo
siete
sigue
siguiente
sin
sino
sobre
sois
sola
solamente
solas
solo
solos
somos
son
soy
su
sus
suya
suyas
suyo
suyos
sí
sólo
tal
también
tampoco
tan
tanto
te
tendremos
tendrá
tendrán
tendrás
tendré
tendréis
tendría
tendríais
tendríamos
tendrían
tendrías
tened
teneis
tenemos
tener
tenga
tengamos
tengan
tengas
tengo
tengáis
tenida
tenidas
tenido
tenidos
teniendo
tenéis
tenía
teníais
teníamos
tenían
tenías
tercera
ti
tiempo
tiene
tienen
tienes
toda
todas
todavía
todo
todos
total
trabaja
trabajais
trabajamos
trabajan
trabajar
trabajas
trabajo
tras
trata
través
tres
tu
tus
tuve
tuviera
tuvierais
tuvieran
tuvieras
tuvieron
tuviese
tuvieseis
tuviesen
tuvieses
tuvimos
tuviste
tuvisteis
tuviéramos
tuviésemos
tuvo
tuya
tuyas
tuyo
tuyos
tú
ultimo
un
una
unas
uno
unos
usa
usais
usamos
usan
usar
usas
uso
usted
va
vais
valor
vamos
van
varias
varios
vaya
veces
ver
verdad
verdadera
verdadero
vez
vosotras
vosotros
voy
vuestra
vuestras
vuestro
vuestros
y
ya
yo
él
éramos
ésta
éstas
éste
éstos
última
últimas
último
últimos'''.split()